In [42]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [39]:
# 데이터 경로 설정
dataset_path = r'C:\Users\impra\Desktop\2024_first_semester\AIUI\Stracker\eye-detection-model\jiwon\drowsy_dataset'

# 데이터 증강을 위한 ImageDataGenerator 설정
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Training 데이터 생성기
train_generator = datagen.flow_from_directory(
    directory=dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

# Validation 데이터 생성기
validation_generator = datagen.flow_from_directory(
    directory=dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 14678 images belonging to 2 classes.
Found 3668 images belonging to 2 classes.


In [43]:
# Pretrained 모델 불러오기 (EfficientNetV2L 예제)
base_model = EfficientNetV2L(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Pretrained 모델의 초기 층들은 고정
for layer in base_model.layers:
    layer.trainable = False

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# 데이터셋 크기 확인
train_steps = train_generator.samples // train_generator.batch_size
validation_steps = validation_generator.samples // validation_generator.batch_size

# 모델 학습
model.fit(
    train_generator,
    steps_per_epoch=train_steps,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    epochs=10  # 에포크 수를 10으로 설정
)

473176280/473176280 ━━━━━━━━━━━━━━━━━━━━ 17s 0us/step
Epoch 1/10
  5/458 ━━━━━━━━━━━━━━━━━━━━ 27:04 4s/step - accuracy: 0.5458 - loss: 0.6778

KeyboardInterrupt: 

In [ ]:
# 모델 저장
model.save('drowsy_detection_model(EffcientnetV2L).h5')